In [1]:
import pandas as pd
from projAll import combined_df, filtered_labels_at_least_5_list
from llama import LLAMA31_8B_LLM

Artifact Id
d3f:Command                      250
d3f:Database                      16
d3f:Software                      16
d3f:HardwareDriver                14
d3f:DisplayServer                 11
d3f:OperatingSystem                8
d3f:FileSystem                     7
d3f:BootLoader                     6
d3f:InterprocessCommunication      5
Name: count, dtype: int64
Artifact Id
d3f:Database                     16
d3f:Software                     16
d3f:Command                      16
d3f:HardwareDriver               14
d3f:DisplayServer                11
d3f:OperatingSystem               8
d3f:FileSystem                    7
d3f:BootLoader                    6
d3f:InterprocessCommunication     5
Name: count, dtype: int64


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\tamar\anaconda3\envs\llm_env\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [2]:
def rephrase(description, author):
    """
    Rephrases the given description as if it were written by the specified author.
    
    Args:
        description (str): The text to be rephrased.
        author (str): The name of the person whose writing style is to be emulated.
        
    Returns:
        str: The rephrased description.
    """
    message = LLAMA31_8B_LLM.invoke(
        f"Rewrite the following description as if it were written by {author['author_title']} who is an expert in {author['expertise']}, without introducing or describing the rewritten text. Provide only the rephrased version: {description}"
    )
    return message.content


In [3]:
authors = [
    {
        "author_title": "Senior Developer",
        "expertise": "Unix commands, software development tools, and debugging utilities."
    },
    {
        "author_title": "Security Researcher",
        "expertise": "Network protocols, interprocess communication, and hardware drivers."
    },
    {
        "author_title": "Database Architect",
        "expertise": "Designing, scaling, and managing distributed and graph databases."
    },
    {
        "author_title": "Software Engineer",
        "expertise": "Developing cross-platform applications, robotics software, and open-source projects."
    },
    {
        "author_title": "Technical Writer",
        "expertise": "Documenting technical tools, APIs, and usage guides for software and hardware."
    },
    {
        "author_title": "Robotics Developer",
        "expertise": "Developing software for robotics, complex systems, and distributed component architectures."
    }
]

In [4]:
df = combined_df
num_of_labels = len(filtered_labels_at_least_5_list)
grouped_df = df.groupby('Artifact Id')
df_dict = {artifact_id: group for artifact_id, group in grouped_df}
label_dfs = {}

for label in filtered_labels_at_least_5_list:
    label_dfs[label] = df_dict.get(label, pd.DataFrame())

In [5]:
new_rows = []  # To store the modified rows

for label, label_df in label_dfs.items():
    print(f"Processing DataFrame for label: {label}")
    for index, row in label_df.iterrows():
        description = row['Example Description']
        for author in authors:
            new_description = rephrase(description, author)
            
            # Create a copy of the row with updated fields
            updated_row = row.copy()
            updated_row['Example Description'] = new_description
            updated_row['Example Source'] = "LLAMA3"
            
            # Add the updated row to the list
            new_rows.append(updated_row)

Processing DataFrame for label: d3f:Command
Processing DataFrame for label: d3f:Database
Processing DataFrame for label: d3f:Software
Processing DataFrame for label: d3f:HardwareDriver
Processing DataFrame for label: d3f:DisplayServer
Processing DataFrame for label: d3f:OperatingSystem
Processing DataFrame for label: d3f:FileSystem
Processing DataFrame for label: d3f:BootLoader
Processing DataFrame for label: d3f:InterprocessCommunication


In [6]:
updated_df = pd.DataFrame(new_rows)
# merged_df = pd.concat([combined_df, updated_df], ignore_index=True)
updated_df.to_csv('csv/dataset_aug.csv', index=False)

In [7]:
print(len(combined_df))
print(len(authors))
print(len(updated_df))

99
6
594


In [8]:
message = LLAMA31_8B_LLM.invoke("write the abc..")
print(message.content)

The ABCs are:

A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y, Z.
